In [3]:
# Import / install relevant Python packages
import numpy as np
import pandas as pd  
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.cluster import (KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN)
from sklearn.neighbors import NearestNeighbors

In [5]:
feature_names_list = ['mom1m','absacc','acc','aeavol','age','agr','baspread',
'beta','betasq','bm','bm_ia','cash','cashdebt','cashpr','cfp','cfp_ia',
'chatoia','chcsho','chempia','chinv','chmom','chpmia','chtx','cinvest',
'convind','currat','depr','divi','divo','dolvol','dy','ear','egr','ep',
'gma','herf','hire','idiovol','ill','indmom','invest','lev','lgr','maxret',
'ms','mve_ia','mvel1','nincr','operprof','pchcapx_ia','pchcurrat','pchdepr',
'pchgm_pchsale','pchquick','pchsale_pchrect','pctacc','pricedelay',
'ps','quick','rd','retvol','roaq','roeq','roic','rsup','salecash',
'salerec','securedind','sgr','sin','sp','std_dolvol','std_turn',
'tang','tb','turn','zerotrade']

mom_names_list = ["mom1m","mom2m","mom3m","mom4m","mom5m","mom6m","mom7m","mom8m","mom9m","mom10m","mom11m","mom12m",
                "mom13m","mom14m","mom15m","mom16m","mom17m","mom18m","mom19m","mom20m","mom21m","mom22m","mom23m","mom24m",
                "mom25m","mom26m","mom27m","mom28m","mom29m","mom30m","mom31m","mom32m","mom33m","mom34m","mom35m","mom36m",
                "mom37m","mom38m","mom39m","mom40m","mom41m","mom42m","mom43m","mom44m","mom45m","mom46m","mom47m","mom48m"]

feature_names_list_full =  feature_names_list + mom_names_list[1:]

PCA_components_number = len(feature_names_list_full)
PCA_components_column_names = ['PCA '+ str(i+1) for i in range(PCA_components_number)]
PCA_components_tresholds_column_names=['n_to_reach_90','n_to_reach_95','n_to_reach_99']
Original_df_columns_names = ['DATE','permno','sic2']
PCA_table_full_columns_names = Original_df_columns_names+PCA_components_column_names+PCA_components_tresholds_column_names
DBSCAN_column_names = ['cluster alpha='+ str(i) for i in range(10,100,10)]

In [3]:
df_PCA = pd.read_csv('D:/USB Drive/Data/DachengXiu/df_PCA_done.csv')

In [4]:
#inicilize df for DBSCAN - make empty df
#initial dataframe
df_DBSCAN=pd.DataFrame(columns=Original_df_columns_names + DBSCAN_column_names)
dates = sorted(df_PCA['DATE'].unique())
for date in dates:
    print("DATE ",date)
    #date = 19791231
    df_DBSCAN_window = df_PCA[df_PCA['DATE'] == date].sort_values('permno')

    #list of column names of PCA components needed to explain 99% of variance
    number_of_PCA_components_for_99_variance_explainability =  df_DBSCAN_window['n_to_reach_99'].min()
    PCA_components_for_99_variance_explainability_column_names = ['PCA '+ str(i+1) for i in range(number_of_PCA_components_for_99_variance_explainability)]

    # Define the key inputs for the DBSCAN model
    min_samples = int(round(np.log(len(df_DBSCAN_window)))) #MinPts is set to be the natural logarithm of the total number of data points N

    # Create an instance of the class NearestNeighbors and fit the model
    #The minimum distance is set as the α percentile of the average distances to the nearest MinPts neighbour points
    #So we have to add 1 since 0 position in distances is 0
    n_neighbors = min_samples + 1 
    #that l1 norm provides more significant and meaningful clustering results for high-dimensional datasets, we choose l1 norm as the distance metric for DBSCAN.
    nn_model    = NearestNeighbors(n_neighbors=n_neighbors, metric='l1')
    neighbors   = nn_model.fit(df_DBSCAN_window[PCA_components_for_99_variance_explainability_column_names])

    # Calcuate average distances to n_neighbors
    distances, indices = neighbors.kneighbors(df_DBSCAN_window[PCA_components_for_99_variance_explainability_column_names])
    distances_avg = distances[:, 1:].mean(axis=1)
    distances_avg = np.sort(distances_avg, axis=0)

    for alpha in range(10,100,10):
        #we use alpha from 10 - 90
        distance_of_alpha_percentile_to_nearest_MinPts = np.percentile(distances_avg, alpha)

        # Create an instance of the class DBSCAN, fit the model with l1 metric
        dbscan_model = DBSCAN(eps=distance_of_alpha_percentile_to_nearest_MinPts,metric='l1',
                                min_samples=min_samples)
        dbscan_model_fited = dbscan_model.fit(df_DBSCAN_window[PCA_components_for_99_variance_explainability_column_names])
        
        #determine cluster lables and fill data to df
        df_DBSCAN_window[f'cluster alpha={alpha}'] = dbscan_model_fited.labels_

    df_DBSCAN=pd.concat([df_DBSCAN,df_DBSCAN_window],axis=0)
df_DBSCAN = df_DBSCAN[Original_df_columns_names+DBSCAN_column_names]


DATE  19791231
DATE  19800131
DATE  19800229
DATE  19800331
DATE  19800430
DATE  19800530
DATE  19800630
DATE  19800731
DATE  19800829
DATE  19800930
DATE  19801031
DATE  19801128
DATE  19801231
DATE  19810130
DATE  19810227
DATE  19810331
DATE  19810430
DATE  19810529
DATE  19810630
DATE  19810731
DATE  19810831
DATE  19810930
DATE  19811030
DATE  19811130
DATE  19811231
DATE  19820129
DATE  19820226
DATE  19820331
DATE  19820430
DATE  19820528
DATE  19820630
DATE  19820730
DATE  19820831
DATE  19820930
DATE  19821029
DATE  19821130
DATE  19821231
DATE  19830131
DATE  19830228
DATE  19830331
DATE  19830429
DATE  19830531
DATE  19830630
DATE  19830729
DATE  19830831
DATE  19830930
DATE  19831031
DATE  19831130
DATE  19831230
DATE  19840131
DATE  19840229
DATE  19840330
DATE  19840430
DATE  19840531
DATE  19840629
DATE  19840731
DATE  19840831
DATE  19840928
DATE  19841031
DATE  19841130
DATE  19841231
DATE  19850131
DATE  19850228
DATE  19850329
DATE  19850430
DATE  19850531
DATE  1985

In [5]:
df_DBSCAN

,DATE,permno,sic2,cluster alpha=10,cluster alpha=20,cluster alpha=30,cluster alpha=40,cluster alpha=50,cluster alpha=60,cluster alpha=70,cluster alpha=80,cluster alpha=90
0,19791231,10006,37.0,0,0,0,0,0,0,0,0,0
1,19791231,10057,36.0,-1,-1,-1,-1,0,0,0,0,0
2,19791231,10137,49.0,0,0,0,0,0,0,0,0,0
3,19791231,10145,99.0,-1,-1,-1,0,0,0,0,0,0
4,19791231,10153,13.0,-1,-1,-1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1502130,20211231,93423,79.0,-1,-1,1,0,0,0,0,0,0
1502131,20211231,93426,36.0,-1,0,1,0,0,0,0,0,0
1502132,20211231,93427,36.0,-1,-1,1,0,0,0,0,0,0
1502133,20211231,93434,1.0,-1,-1,-1,-1,-1,-1,-1,0,0


In [ ]:
#df_DBSCAN.to_csv('D:/USB Drive/Data/DachengXiu/df_DBSCAN.csv')

Analysis:

In [6]:
df_DBSCAN = pd.read_csv('D:/USB Drive/Data/DachengXiu/df_DBSCAN.csv')

In [7]:
df_1990_2000 = df_DBSCAN[(df_DBSCAN['DATE'] > 19891231 ) & (df_DBSCAN['DATE'] < 20210101)]

In [8]:
#Number of clusters 2
df_1990_2000.groupby(['DATE','cluster alpha=10']).nunique()['permno'].count()/372-1
#-1 are outliers
#1.54

1.5403225806451615

In [9]:
#Number of stocks in total 3157
df_1990_2000.groupby(['DATE'])['permno'].count().mean()
#3286

3286.5779569892475

In [10]:
#Number of stocks in clusters 376 (12.05)
df_1990_2000[df_1990_2000['cluster alpha=10'] > -1].groupby(['DATE'])['permno'].count().mean()
#429

429.22849462365593

In [11]:
#Number of outliers 2781 (87.95)
-df_1990_2000[DBSCAN_column_names].sum()/372
#In paper we use cluster alpha=10    
# 2847.325269

cluster alpha=10    2847.325269
cluster alpha=20    2448.150538
cluster alpha=30    2077.954301
cluster alpha=40    1727.026882
cluster alpha=50    1392.755376
cluster alpha=60    1079.911290
cluster alpha=70     792.344086
cluster alpha=80     517.263441
cluster alpha=90     247.180108
dtype: float64

In [14]:
#Number of stocks in the biggest cluster 336 (88.89)
df_1990_2000[df_1990_2000['cluster alpha=10'] == 0].groupby(['DATE','cluster alpha=10']).nunique()['permno'].mean()
#420.1317204301075

420.1317204301075

In [15]:
#Number of stocks in the second biggest cluster 32 (8.74)
df_1990_2000[df_1990_2000['cluster alpha=10'] == 1].groupby(['DATE','cluster alpha=10']).nunique()['permno'].mean()
#.sort_values(ascending=False).head(5)
#18.82

18.822085889570552

In [16]:
#Number of stocks in the third biggest cluster 6 (1.88) 
df_1990_2000[df_1990_2000['cluster alpha=10'] == 2].groupby(['DATE','cluster alpha=10']).nunique()['permno'].mean()
#.sort_values(ascending=False).head(5)
#8.58

8.588235294117647